In [63]:
%pip install "accelerate>=1.10.1" "datasets>=4.0.0" "huggingface-hub>=0.34.4" "ipykernel>=6.30.1" "ipywebrtc>=0.6.0" "ipywidgets>=8.1.7" "librosa>=0.11.0" "pip>=25.2" "snac>=1.2.1" "soundfile>=0.13.1" "torch>=2.8.0" "torchaudio>=2.8.0" "transformers>=4.56.1" "wandb>=0.21.4" 

Note: you may need to restart the kernel to use updated packages.


In [64]:
from datasets import load_dataset

name_to_push_dataset_to = "cubbk/rixvox-tokenised"


df = load_dataset("cubbk/rixvox_parquet_dataset")["validation"] # type: ignore

df

Dataset({
    features: ['speaker', 'text', 'waveform', 'sampling_rate'],
    num_rows: 7933
})

In [65]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import torch
from snac import SNAC


model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz")
model.to("cuda")

SNAC(
  (encoder): Encoder(
    (block): Sequential(
      (0): ParametrizedConv1d(
        1, 48, kernel_size=(7,), stride=(1,), padding=(3,)
        (parametrizations): ModuleDict(
          (weight): ParametrizationList(
            (0): _WeightNorm()
          )
        )
      )
      (1): EncoderBlock(
        (block): Sequential(
          (0): ResidualUnit(
            (block): Sequential(
              (0): Snake1d()
              (1): ParametrizedConv1d(
                48, 48, kernel_size=(7,), stride=(1,), padding=(3,), groups=48
                (parametrizations): ModuleDict(
                  (weight): ParametrizationList(
                    (0): _WeightNorm()
                  )
                )
              )
              (2): Snake1d()
              (3): ParametrizedConv1d(
                48, 48, kernel_size=(1,), stride=(1,)
                (parametrizations): ModuleDict(
                  (weight): ParametrizationList(
                    (0): _WeightNorm()
    

In [66]:
#@title Tokenisation Function
import torchaudio.transforms as T
from datasets import Audio

def tokenise_audio(waveform, sp):
  waveform = torch.from_numpy(waveform).unsqueeze(0)
  waveform = waveform.to(dtype=torch.float32)
  resample_transform = T.Resample(orig_freq=sp, new_freq=24000)
  waveform = resample_transform(waveform)
  waveform = waveform.unsqueeze(0).to("cuda")
  #generate the codes from snac
  with torch.inference_mode():
    codes = model.encode(waveform)

  all_codes = []
  for i in range(codes[0].shape[1]):
    all_codes.append(codes[0][0][i].item()+128266)
    all_codes.append(codes[1][0][2*i].item()+128266+4096)
    all_codes.append(codes[2][0][4*i].item()+128266+(2*4096))
    all_codes.append(codes[2][0][(4*i)+1].item()+128266+(3*4096))
    all_codes.append(codes[1][0][(2*i)+1].item()+128266+(4*4096))
    all_codes.append(codes[2][0][(4*i)+2].item()+128266+(5*4096))
    all_codes.append(codes[2][0][(4*i)+3].item()+128266+(6*4096))


  return all_codes

In [67]:
def add_codes(example):
    wf = np.asarray(example["waveform"], dtype=np.float32)  # ensure ndarray
    example["codes_list"] = tokenise_audio(wf, example["sampling_rate"])
    return example

ds = df.map(add_codes, remove_columns=["waveform","sampling_rate"])

Map:   0%|          | 0/7933 [00:00<?, ? examples/s]

In [68]:
#@title Load Tokenizer
tokeniser_length = 128256
start_of_text = 128000
end_of_text = 128009

start_of_speech = tokeniser_length + 1
end_of_speech = tokeniser_length + 2

start_of_human = tokeniser_length + 3
end_of_human = tokeniser_length + 4

start_of_ai = tokeniser_length + 5
end_of_ai =  tokeniser_length + 6
pad_token = tokeniser_length + 7

audio_tokens_start = tokeniser_length + 10

tokenizer_name = "canopylabs/orpheus-3b-0.1-pretrained"

from transformers import AutoTokenizer
import os
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
num_proc = os.cpu_count() - 2

for example in ds:
    print(example["text"])
    print(example["codes_list"])

ds = ds.filter(lambda x: x["codes_list"] is not None)
ds = ds.filter(lambda x: len(x["codes_list"]) > 0)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Inom det största vårdföretaget Attendo är de större privata ägarna män. Med få undantag är det män i toppen på de största vinstdrivande bolagen inom välfärden. Vinstjakten som incitament inom offentlig verksamhet måste stoppas, samtidigt som man ställer krav på de privata aktörer som finns inom välfärden. Genom att reglera vinsterna i välfärden kan man skapa förutsättningar för att säkra kvalitet och långsiktighet.
[130015, 136331, 137704, 143438, 147738, 149696, 153907, 129096, 136056, 140316, 142845, 145287, 152307, 153463, 130683, 135450, 136675, 141170, 147325, 152163, 154428, 130646, 133094, 138523, 141022, 148418, 150345, 155276, 129982, 133805, 138065, 140871, 146075, 150058, 155515, 128996, 133143, 137226, 141350, 145814, 149992, 155138, 131183, 133906, 138146, 142988, 147889, 149374, 152854, 131601, 134491, 139817, 143065, 146114, 151992, 153416, 128426, 135074, 139652, 143239, 145698, 149110, 152934, 130557, 136147, 137018, 143412, 147591, 152720, 154272, 130948, 136088, 1389

Filter:   0%|          | 0/7933 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7933 [00:00<?, ? examples/s]

In [69]:
#@title Create Input Ids
def remove_duplicate_frames(example):
    vals = example["codes_list"]
    if len(vals) % 7 != 0:
        raise ValueError("Input list length must be divisible by 7")

    result = vals[:7]

    removed_frames = 0

    for i in range(7, len(vals), 7):
        current_first = vals[i]
        previous_first = result[-7]

        if current_first != previous_first:
            result.extend(vals[i:i+7])
        else:
            removed_frames += 1

    example["codes_list"] = result

    return example

ds = ds.map(remove_duplicate_frames, num_proc=num_proc)

tok_info = '''*** HERE you can modify the text prompt
i.e. if you wanted a multispeaker model like canopylabs/orpheus-3b-0.1-ft, you can pass:
f"{example["source"]}:  {example["text"]}", as is passed.
'''
print(tok_info)

def create_input_ids(example):
    text_ids = tokenizer.encode(example["text"],  add_special_tokens=True)
    text_ids.append(end_of_text)
    example["text_tokens"] = text_ids
    input_ids = (
        [start_of_human]
        + example["text_tokens"]
        + [end_of_human]
        + [start_of_ai]
        + [start_of_speech]
        + example["codes_list"]
        + [end_of_speech]
        + [end_of_ai]
    )
    example["input_ids"] = input_ids
    example["labels"] = input_ids
    example["attention_mask"] = [1] * len(input_ids)

    return example

ds = ds.map(create_input_ids, num_proc=num_proc, remove_columns=["text", "codes_list"])

ds


Map (num_proc=254):   0%|          | 0/7933 [00:00<?, ? examples/s]

*** HERE you can modify the text prompt
i.e. if you wanted a multispeaker model like canopylabs/orpheus-3b-0.1-ft, you can pass:
f"{example["source"]}:  {example["text"]}", as is passed.



Map (num_proc=254):   0%|          | 0/7933 [00:00<?, ? examples/s]

Dataset({
    features: ['speaker', 'text_tokens', 'input_ids', 'labels', 'attention_mask'],
    num_rows: 7933
})

In [70]:
columns_to_keep = ["input_ids", "labels", "attention_mask"]
columns_to_remove = [col for col in ds.column_names if col not in columns_to_keep]

ds = ds.remove_columns(columns_to_remove)


In [72]:
ds.push_to_hub(name_to_push_dataset_to, data_dir="dev")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  56%|#####6    | 33.5MB / 59.4MB            

README.md:   0%|          | 0.00/355 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/cubbk/rixvox-tokenised/commit/609788cbfd7514c82a91e3c5800cd0ee6c908e23', commit_message='Upload dataset', commit_description='', oid='609788cbfd7514c82a91e3c5800cd0ee6c908e23', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/cubbk/rixvox-tokenised', endpoint='https://huggingface.co', repo_type='dataset', repo_id='cubbk/rixvox-tokenised'), pr_revision=None, pr_num=None)